In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the data
jobreq_data = pd.read_csv("cleaned_vacancies_data.csv")
cv_data = pd.read_csv("cleaned_cv_data.csv")

Translate the datasets first to avoid biased points of similarities

In [ ]:
from googletrans import Translator
from langdetect import detect
# Initialize the translator
translator = Translator()

# Function to detect and translate text
def translate_text(text):
    try:
        # Detect the language
        lang = detect(text)
        # Translate only if the text is in Indonesian
        if lang == 'id':
            translated = translator.translate(text, src='id', dest='en')
            return translated.text
        else:
            # Return the original text if it's not in Indonesian
            return text
    except Exception as e:
        # If detection or translation fails, return the original text
        return text
# Translate jobreq_data descriptions
jobreq_data['description'] = jobreq_data['description'].apply(translate_text)

# Translate cv_data text
cv_data['Text'] = cv_data['Text'].apply(translate_text)

calculate the similarity of the jobreq (job requirement) dataset and the cv's dataset

In [3]:
# Initialize the TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Concatenate all descriptions and texts to fit the vectorizer
all_texts = pd.concat([jobreq_data['description'], cv_data['Text']])
vectorizer.fit(all_texts)

# Transform the job descriptions and CV texts to their TF-IDF representation
jobreq_tfidf = vectorizer.transform(jobreq_data['description'])
cv_tfidf = vectorizer.transform(cv_data['Text'])

# Calculate cosine similarity using TensorFlow
def cosine_similarity_tf(tfidf_matrix1, tfidf_matrix2):
    tfidf_matrix1 = tf.constant(tfidf_matrix1.toarray(), dtype=tf.float32)
    tfidf_matrix2 = tf.constant(tfidf_matrix2.toarray(), dtype=tf.float32)
    dot_product = tf.linalg.matmul(tfidf_matrix1, tfidf_matrix2, transpose_b=True)
    norm_matrix1 = tf.sqrt(tf.reduce_sum(tf.square(tfidf_matrix1), axis=1, keepdims=True))
    norm_matrix2 = tf.sqrt(tf.reduce_sum(tf.square(tfidf_matrix2), axis=1, keepdims=True))
    norm_product = tf.linalg.matmul(norm_matrix1, norm_matrix2, transpose_b=True)
    similarity = dot_product / norm_product
    return similarity

In [4]:
# Calculate the similarity
similarity_matrix = cosine_similarity_tf(jobreq_tfidf, cv_tfidf)
similarity_matrix_np = similarity_matrix.numpy()

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix_np, index=jobreq_data.index, columns=cv_data.index)

In [5]:
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,418,419,420,421,422,423,424,425,426,427
0,0.012785,0.010183,0.069744,0.020618,0.004395,0.014869,0.016606,0.010769,0.021189,0.030767,...,0.045073,0.043696,0.038822,0.024304,0.001572,0.035637,0.013906,0.006442,0.024702,0.045309
1,0.030399,0.007897,0.021916,0.026240,0.017771,0.003152,0.034305,0.034525,0.047479,0.052966,...,0.046756,0.041399,0.054001,0.039430,0.029149,0.025855,0.014119,0.020057,0.026980,0.063277
2,0.024856,0.006835,0.022732,0.023619,0.014639,0.000000,0.028282,0.035462,0.037284,0.044950,...,0.033871,0.029847,0.040090,0.023334,0.023786,0.021655,0.012975,0.013563,0.020683,0.047165
3,0.099789,0.021469,0.085886,0.041869,0.031772,0.001258,0.076088,0.037267,0.054755,0.080897,...,0.121026,0.073477,0.081918,0.060860,0.116674,0.077738,0.039035,0.052128,0.068214,0.060701
4,0.012785,0.010183,0.069744,0.020618,0.004395,0.014869,0.016606,0.010769,0.021189,0.030767,...,0.045073,0.043696,0.038822,0.024304,0.001572,0.035637,0.013906,0.006442,0.024702,0.045309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,0.074922,0.006761,0.074653,0.037524,0.036242,0.006148,0.081610,0.020786,0.055177,0.081846,...,0.130355,0.068830,0.093960,0.067039,0.087012,0.089252,0.059586,0.048852,0.045361,0.074616
493,0.011585,0.023993,0.069925,0.022286,0.000865,0.009263,0.005133,0.023320,0.015202,0.011388,...,0.016599,0.002688,0.061566,0.030246,0.002772,0.041805,0.020578,0.006270,0.018294,0.015421
494,0.068623,0.008464,0.070078,0.054196,0.027784,0.000000,0.084466,0.049908,0.044837,0.095100,...,0.114410,0.088784,0.097626,0.066425,0.077225,0.071418,0.071418,0.063214,0.103046,0.099639
495,0.060338,0.009903,0.055633,0.051680,0.024584,0.000000,0.053376,0.048128,0.033652,0.061879,...,0.104051,0.086678,0.096230,0.057891,0.081210,0.055716,0.037184,0.058508,0.080777,0.076223


Load the library for training the collaborative filtering model for candidate ranking

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dot, Flatten
from tensorflow.keras.models import Model

In [7]:
# Load the similarity matrix
similarity_matrix_np = similarity_matrix.numpy()

# Define function to prepare data for collaborative filtering
def prepare_data(similarity_matrix):
    # Get the indices of the top recommendations for each job
    top_candidates_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1]
    # Create DataFrame to hold the user-item interactions
    interactions = []
    for job_idx, top_candidates in enumerate(top_candidates_indices):
        for candidate_idx in top_candidates:
            interactions.append([job_idx, candidate_idx, similarity_matrix[job_idx, candidate_idx]])
    cv_data_with_points = pd.DataFrame(interactions, columns=['job_idx', 'candidate_idx', 'points'])
    return cv_data_with_points

In [8]:
# Prepare data for collaborative filtering
cv_data_with_points = prepare_data(similarity_matrix_np)

# Convert indices to continuous IDs for embedding purposes
cv_data_with_points['job_idx'] = cv_data_with_points['job_idx'].astype('category').cat.codes
cv_data_with_points['candidate_idx'] = cv_data_with_points['candidate_idx'].astype('category').cat.codes

# Number of unique job requirements and candidates
n_jobs = cv_data_with_points['job_idx'].nunique()
n_candidates = cv_data_with_points['candidate_idx'].nunique()

# Define the input layers
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

# Define the embedding layers
user_embedding = Embedding(n_candidates, 20, embeddings_initializer='uniform', name='user_embedding')(user_input)
item_embedding = Embedding(n_jobs, 20, embeddings_initializer='uniform', name='item_embedding')(item_input)

# Flatten the embedding layers
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

# Compute the dot product of the embeddings
y = Dot(axes=1)([user_vecs, item_vecs])

# Create the model
model = Model(inputs=[user_input, item_input], outputs=y)
model.compile(optimizer='adam', loss='mse')

# Prepare the data for training
X = cv_data_with_points[['candidate_idx', 'job_idx']].values
y = cv_data_with_points['points'].values

# Train the model
model.fit([X[:, 0], X[:, 1]], y, epochs=128, batch_size=128, verbose=1)

Epoch 1/128
1662/1662 [==============================] - 4s 2ms/step - loss: 0.0015
Epoch 2/128
1662/1662 [==============================] - 3s 2ms/step - loss: 2.5482e-04
Epoch 3/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.8786e-04
Epoch 4/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.6599e-04
Epoch 5/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.5935e-04
Epoch 6/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.5706e-04
Epoch 7/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.5554e-04
Epoch 8/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.5478e-04
Epoch 9/128
1662/1662 [==============================] - 4s 2ms/step - loss: 1.5466e-04
Epoch 10/128
1662/1662 [==============================] - 4s 2ms/step - loss: 1.5427e-04
Epoch 11/128
1662/1662 [==============================] - 3s 2ms/step - loss: 1.5349e-04
Epoch 12/128
1662/1662 [==========

In [9]:
# Save the model to an .h5 file
model.save('collaborative_filtering_model.h5')

C:\Users\ASUS\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
from tensorflow.keras.models import load_model
# Load the saved model
model = load_model('collaborative_filtering_model.h5')

Make a recommendation function to print the recommended candidate's cv

In [12]:
# Define a function to recommend CVs for a job requirement using the trained collaborative filtering model
def recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data, top_n=5):
    # Transform the job description to its TF-IDF representation
    jobreq_tfidf = vectorizer.transform([jobreq_data.loc[jobreq_id, 'description']])
    # Get the indices of CVs
    cv_indices = np.arange(len(cv_data))
    # Repeat the job requirement index for all CVs
    job_indices = np.repeat(jobreq_id, len(cv_data))
    # Predict the points using the model
    predicted_points = model.predict([cv_indices, job_indices])
    # Get the top recommended CVs
    top_cvs_indices = predicted_points.squeeze().argsort()[::-1][:top_n]
    # Get the CV texts and similarity scores
    top_cvs_with_scores = []
    for cv_idx in top_cvs_indices:
        cv_text = cv_data.loc[cv_idx, 'Text']
        similarity_score = predicted_points[cv_idx]
        top_cvs_with_scores.append((cv_text, similarity_score))
    return top_cvs_with_scores

In [17]:
# Example: get top 5 CVs for a specific job requirement using the collaborative filtering model
jobreq_id = 0  # Replace with a job requirement index for testing

# you can print the code below to check the regarding jobreq_text
#jobreq_text = jobreq_data.iloc[jobreq_id]['description']
#print(jobreq_text)
top_cvs_with_scores = recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data)

print("Top CVs for Job Requirement ID {}: ".format(jobreq_id))
for cv_text, similarity_score in top_cvs_with_scores:
    print(f"CV: {cv_text}\nSimilarity Score: {similarity_score}\n")

14/14 [==============================] - 0s 2ms/step
Top CVs for Job Requirement ID 0: 
CV: ni ketut alit astuti ef kota kupang indonesia 85116 081266929973 e niketutaastutigmailcom professional agile impactful high dedication value live today becoming summary dedicated mentor program director experience achieving tangible result crossteam collaboration proactive excited partner likeminded individual achieve goal core qualification ee formal education training « personal development technique interactive teachinglearning public speaking program designer program management inspiring leadership team management project coordination sekolah tinggi iimu manajemen kupang kupang ntt 2022 bachelor economics finance management ministry tourism creative economy indonesia 2021 certificate compliment t0t startup create class batch 1 bekup ministry tourism creative economy becoming startup digital trainer undp indonesia 2020 certificate participation business integrity training undp undp collaborat